In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
# Download the dataset
import kagglehub
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# Replace with your dataset name
path = kagglehub.dataset_download("smagnan/1-million-reddit-comments-from-40-subreddits") # To directly download from kaggle website
# path = "/root/.cache/kagglehub/datasets/smagnan/1-million-reddit-comments-from-40-subreddits/versions/1"


100%|██████████| 71.2M/71.2M [00:00<00:00, 125MB/s]

Extracting files...


In [ ]:
# Load CSV File (Modify file name if needed)
df = pd.read_csv(f"{path}/kaggle_RC_2019-05.csv")
# df = pd.read_csv("kaggle_RC_2019-05.csv")
print(df.shape)
print(df.head())

(1000000, 4)
       subreddit                                               body  \
0  gameofthrones  Your submission has been automatically removed...   
1            aww  Dont squeeze her with you massive hand, you me...   
2         gaming  It's pretty well known and it was a paid produ...   
3           news  You know we have laws against that currently c...   
4       politics  Yes, there is a difference between gentle supp...   

   controversiality  score  
0                 0      1  
1                 0     19  
2                 0      3  
3                 0     10  
4                 0      1  


In [ ]:
df_new = df.sample(n=30000, random_state=44).reset_index(drop=True)

In [ ]:
# Initialize counters
count_equal_1 = 0
count_above_1 = 0
count_below_1 = 0

In [ ]:
# Count entries where score == 1
count_equal_1 = (df_new['score'] == 1).sum()

# Count entries where score > 1
count_above_1 = (df_new['score'] > 1).sum()

# Count entries where score < 1
count_below_1 = (df_new['score'] < 1).sum()

In [ ]:
# Print results
print("Number of values > 1:", count_above_1)
print("Number of values < 1:", count_below_1)
print("Number of values == 1:", count_equal_1)

Number of values > 1: 15685
Number of values < 1: 2750
Number of values == 1: 11565


In [ ]:
sum = count_above_1 + count_below_1 + count_equal_1

print("Above one percent = ", ((count_above_1/sum) * 100))

Above one percent =  52.28333333333334


In [ ]:
print("Below one percent = ", ((count_below_1/sum) * 100))

Below one percent =  9.166666666666666


In [ ]:
print("Equals one percent = ", ((count_equal_1/sum) * 100))

Equals one percent =  38.550000000000004


---------------------------------------------------------------------------

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer

In [ ]:
# Convert score into class labels
def convert_score(score):
    if score < 1:
        return 0  # Negative
    elif score == 1:
        return 1  # Neutral
    else:
        return 2  # Positive

In [ ]:
df_new['label'] = df_new['score'].apply(convert_score)

In [ ]:
df_new.head(10)

,subreddit,body,controversiality,score,label
0,unpopularopinion,basketball and soccer? wtf,0,2,2
1,apexlegends,Once you finish the battle pass which is aroun...,0,1,1
2,soccer,&gt; Same with lahoz before yesterday's game \...,0,9,2
3,RoastMe,Your fingernails are nasty 🤢,0,1,1
4,soccer,It’s easier to be blunt when you are an attrac...,0,6,2
5,relationship_advice,I have gone through similar stuff. I was moles...,0,2,2
6,SquaredCircle,/s?\n\nNo way you can be serious.,0,1,1
7,nfl,When love is in full bloom you gotta take the ...,0,3,2
8,gameofthrones,i asked this right afte rthe ep too! thought m...,0,1,1
9,gameofthrones,You think white walkers/others are the same too?,0,0,0


In [ ]:
# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_new['body'].tolist(),
    df_new['label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [ ]:
# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class RedditDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = RedditDataset(train_texts, train_labels, tokenizer)
val_dataset = RedditDataset(val_texts, val_labels, tokenizer)

In [ ]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# import collections

# # Count the occurrences of each label using collections.Counter
# label_counts = collections.Counter(train_labels)

# # Compute inverse frequency weights
# weights = []
# for i in range(3):  # assuming 3 classes: 0, 1, 2
#     count = label_counts[i]
#     weight = num_samples / (3.0 * count)  # balanced inverse frequency
#     weights.append(weight)

# # Convert to tensor
# class_weights = torch.tensor(weights, dtype=torch.float32)

# # Normalize weights so they sum to 1
# class_weights = class_weights / class_weights.sum()
# print(class_weights)


In [ ]:
# from transformers import RobertaConfig

# config = RobertaConfig.from_pretrained('roberta-base', num_labels=3)

# class WeightedRoberta(RobertaForSequenceClassification):
#     def __init__(self, config, weight_tensor):
#         super().__init__(config)
#         self.weight_tensor = weight_tensor

#     def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
#         outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
#         if labels is not None:
#             loss_fct = nn.CrossEntropyLoss(weight=self.weight_tensor)
#             loss = loss_fct(outputs.logits, labels)
#             outputs.loss = loss
#         return outputs

#   model = WeightedRoberta.from_pretrained('roberta-base', config=config, weight_tensor=weights)



In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    # evaluation_strategy='epoch',
    report_to='none',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:

# predictions = trainer.predict(val_dataset)
# predicted_labels = predictions.predictions.argmax(-1)



In [ ]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# report = classification_report(val_labels, predicted_labels)
# print(report)

# Predict on validation set
predictions = trainer.predict(val_dataset)

# Extract logits and true labels
logits = predictions.predictions
y_pred = logits.argmax(axis=1)
y_true = predictions.label_ids

# # Print full classification report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"]))




# ... (Your existing code to generate classification_report) ...

# Get the classification report as a dictionary
report_dict = classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"], output_dict=True)

# Extract relevant metrics
metrics = ['precision', 'recall', 'f1-score']
classes = ['Negative', 'Neutral', 'Positive']
data = np.array([[report_dict[c][m] for m in metrics] for c in classes])

# Create the plot
fig, ax = plt.subplots(figsize=(10, 5))
width = 0.2  # Width of each bar
x = np.arange(len(classes))

# Plot bars for each metric
rects1 = ax.bar(x - width, data[:, 0], width, label='Precision')
rects2 = ax.bar(x, data[:, 1], width, label='Recall')
rects3 = ax.bar(x + width, data[:, 2], width, label='F1-score')

# Set labels, title, and legend
ax.set_ylabel('Scores')
ax.set_title('Classification Report')
ax.set_xticks(x)
ax.set_xticklabels(classes)
ax.legend()

# Add value labels on top of bars
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Function to predict sentiment from user input
def predict_user_input(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred = torch.argmax(logits, dim=1).item()

    label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return label_mapping[pred]

# Take input from user
user_text = input("\nEnter a comment for sentiment prediction: ")

#  Predict label
predicted_label = predict_user_input(user_text, model, tokenizer, device)

# Print output
print(f"\nPredicted Sentiment: {predicted_label}")
